# Converting SCEN7 files to SCEN

Hacky notebook which may have some useful bits but shouldn't ever appear in master of pymagicc in this form.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint
from copy import deepcopy

import pandas as pd
import numpy as np
import pyam
from pyam.utils import LONG_IDX
import pint
from pint.pandas_interface import PintArray
from pint.errors import DimensionalityError

import pymagicc
from pymagicc.io import MAGICCData

import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [2]:
scen7_path = "/Users/zebedeenicholls/Downloads/drive-download-20181003T174949Z-001/"

In [3]:
def make_total_vars(mdata):
    variables = mdata.df.columns.get_level_values("VARIABLE")
    to_append = []
    for var in variables:
        pair_var = var.replace("I_", "B_")
        if (pair_var in variables) and (pair_var != var):
            var_df = mdata[var].copy()
            var_df.columns = var_df.columns.droplevel('VARIABLE')

            pair_var_df = mdata[pair_var].copy()
            pair_var_df.columns = pair_var_df.columns.droplevel('VARIABLE')

            sum_df = var_df + pair_var_df
            var_base = var.split("_")[0][:-1]
            total_var = "_".join([var_base, "EMIS"])
            sum_df = pd.concat([sum_df], keys=[total_var], names=["VARIABLE"], axis=1)
            to_append.append(sum_df)

    mdata.df = pd.concat([mdata.df] + to_append, axis=1)
    
    return mdata

def make_world_totals(mdata):
    variables = mdata.df.columns.get_level_values("VARIABLE").unique()
    to_append = []
    for var in variables:
        var_df = mdata[var]
        if "WORLD" in var_df.columns.get_level_values("REGION"):
            continue
            
        world_df = pd.DataFrame(var_df.sum(axis=1)).copy()
        
        world_df.columns = pd.MultiIndex.from_arrays(
            [
                [var_df.columns.get_level_values("VARIABLE")[0]], 
                [var_df.columns.get_level_values("TODO")[0]], 
                [var_df.columns.get_level_values("UNITS")[0]], 
                ["WORLD"]
            ],
            names=["VARIABLE", "TODO", "UNITS", "REGION"]
        )

        to_append.append(world_df)

    mdata.df = pd.concat([mdata.df] + to_append, axis=1)
    
    return mdata

def convert_scen7_to_scen_hack(scen_file_path, scen_file_name):
    mdata = MAGICCData()
    mdata.read(filepath=scen_file_path, filename=scen_file_name)
    
    mdatatwo = MAGICCData()
    mdatatwo.read(filepath=scen_file_path, filename=scen_file_name.replace("ERTH90026", "ERTH90026_HALO"))
    
#     mdatathree = MAGICCData()
#     mdatathree.read(filepath=scen_file_path, filename=scen_file_name.replace("WORLD_Ext2250.SCEN7", "HFC_C2F6_CF4_SF6_MISSGAS_Ext2250.SCEN7"))
    
    mdata.df = pd.concat([mdata.df, mdatatwo.df], axis=1)
    
    mdata = make_total_vars(mdata)

    expect_vars = [v + "_EMIS" for v in pymagicc.definitions.scen_emms_code_1]
    mdata.df = mdata.iloc[
        :, 
        mdata.columns.get_level_values("VARIABLE").isin(expect_vars)
    ].dropna().T.drop_duplicates().T
    
    mdata = make_world_totals(mdata)
    
    mdata.df = mdata["WORLD"]
    
#     regions = mdata.df.columns.get_level_values("REGION").tolist()
#     for i, level in enumerate(mdata.df.columns.levels):
#         if level.name == "REGION":
#             regions = mdata.df.columns.levels[i]
#             break
#     region_mappings = {
#         "R6OECD90": "R5OECD",
#         "R6ASIA": "R5ASIA",
#         "R6MAF": "R5MAF",
#         "R6LAM": "R5LAM",
#         "R6REF": "R5REF",
#     }
#     for region in region_mappings:
#         regions = [
#             region_mappings[r] 
#             if r in region_mappings else r 
#             for r in regions
#         ]
    
#     mdata.df.columns = mdata.df.columns.set_levels(
#         regions,
#         level="REGION",
#     )

    mdata.write(scen_file_name.replace("SCEN7", "SCEN"), scen_file_path)

In [6]:
for f in listdir(scen7_path):
    if ("HALO" in f) or (not f.endswith("SCEN7")):
        continue
    convert_scen7_to_scen_hack(scen7_path, f)